# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [3]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # TODO: Implement Function
    source_id_text = [[source_vocab_to_int.get(word) for word in line.split()] for line in source_text.split('\n')]
    target_id_text = [[target_vocab_to_int.get(word) for word in line.split()] + 
                      [target_vocab_to_int['<EOS>']] for line in target_text.split('\n')]
    
    print (source_id_text[:5])
    print (target_id_text[:5])
    return source_id_text, target_id_text

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

[[28, 6, 24, 16, 22, 26, 29, 15, 10, 30, 24, 23, 4, 18, 25], [27, 31, 13, 24, 12, 21, 26, 8, 15, 10, 30, 24, 12, 14, 4, 11, 25], [5, 24, 12, 22, 26, 7, 15, 10, 30, 24, 12, 19, 4, 32, 25], [27, 31, 13, 24, 16, 20, 26, 32, 15, 10, 30, 24, 9, 4, 17, 25]]
[[31, 6, 16, 14, 10, 7, 26, 4, 15, 32, 33, 16, 21, 28, 8, 25, 1], [27, 24, 16, 17, 9, 28, 11, 15, 32, 33, 22, 12, 28, 29, 25, 1], [5, 16, 17, 10, 28, 18, 15, 32, 33, 16, 17, 20, 28, 19, 25, 1], [27, 24, 16, 14, 13, 28, 19, 15, 32, 33, 30, 9, 28, 23, 25, 1]]
Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

[[225, 121, 159, 29, 102, 222, 214, 205, 82, 171, 159, 103, 6, 147, 52], [167, 134, 201, 159, 24, 40, 222, 191, 205, 82, 171, 159, 24, 140, 6, 60, 52], [9, 159, 24, 102, 222, 14, 205, 82, 171, 159, 24, 33, 6, 173, 52], [167, 134, 201, 159, 29, 34, 222, 173, 205, 82, 171, 159, 17, 6, 30, 52], [114, 209, 97, 183, 159, 167, 139, 205, 11, 32, 209, 97, 159, 167, 188, 52]]
[[354, 137, 236, 126, 259, 6, 18, 156, 83, 204, 117, 236, 63, 66, 7, 111, 1], [200, 309, 236, 11, 54, 66, 214, 83, 204, 117, 273, 263, 66, 353, 111, 1], [185, 236, 11, 259, 66, 168, 83, 204, 117, 236, 11, 245, 66, 349, 111, 1], [200, 309, 236, 126, 82, 66, 349, 83, 204, 117, 67, 54, 66, 275, 111, 1], [221, 325, 188, 336, 236, 286, 109, 83, 337, 149, 325, 188, 236, 147, 238, 111, 1]]


# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper
import problem_unittests as tests

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0


E:\Anaconda3\envs\trans\lib\site-packages\ipykernel_launcher.py:15: UserWarning: No GPU found. Please use a GPU to train your neural network.
  from ipykernel import kernelapp as app


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [3]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # TODO: Implement Function
    input_text = tf.placeholder(tf.int32, [None, None], name = 'input')
    targets = tf.placeholder(tf.int32, [None, None])
    learning_rate = tf.placeholder(tf.float32)
    keep_prob = tf.placeholder(tf.float32, name = 'keep_prob')
    target_seq_len = tf.placeholder(tf.int32, [None], name = 'target_sequence_length')
    max_tar_len = tf.reduce_max(target_seq_len, name = 'max_target_len')
    source_seq_len = tf.placeholder(tf.int32, [None], name = 'source_sequence_length')
    
    return input_text, targets, learning_rate, keep_prob, target_seq_len, max_tar_len, source_seq_len


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [4]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # TODO: Implement Function
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size, 1], target_vocab_to_int['<GO>']), ending], 1)
    return dec_input

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [5]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # TODO: Implement Function
    #embed the encoder
    enc_embed_input = tf.contrib.layers.embed_sequence(rnn_inputs,
                                                       source_vocab_size,
                                                       encoding_embedding_size)
    # wrap RNN cell with dropout (made initializer a seperate object according to suggestion)
    initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2)
    def make_wrapped_cell(rnn_size, keep_prob):
        enc_cell = tf.contrib.rnn.LSTMCell(rnn_size, initializer=initializer)
        enc_cell = tf.contrib.rnn.DropoutWrapper(enc_cell, output_keep_prob=keep_prob)
        return enc_cell
    
    # stack cell
    enc_cell = tf.contrib.rnn.MultiRNNCell([make_wrapped_cell(rnn_size, keep_prob) for _ in range(num_layers)])
    
    RNN_output, RNN_state = tf.nn.dynamic_rnn(enc_cell, enc_embed_input, sequence_length=source_sequence_length, dtype=tf.float32)
    
    return RNN_output, RNN_state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [6]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # TODO: Implement Function
    training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                                        sequence_length=target_sequence_length,
                                                        time_major=False)
        
        
    # Basic decoder with dropout
    dec_cell = tf.contrib.rnn.DropoutWrapper(dec_cell, 
                                             output_keep_prob=keep_prob)
    
    training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                       training_helper,
                                                       encoder_state,
                                                       output_layer)
    
        
    # Perform dynamic decoding using the decoder
    training_decoder_output = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                                impute_finished=True,
                                                                maximum_iterations=max_summary_length)[0]
    
    return training_decoder_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [7]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # TODO: Implement Function
    # Helper for the inference process.
    start_tokens = tf.tile(tf.constant([start_of_sequence_id], dtype=tf.int32), 
                           [batch_size], 
                           name='start_tokens')
    inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings,
                                                                start_tokens,
                                                                end_of_sequence_id)

    # Basic decoder
    dec_cell = tf.contrib.rnn.DropoutWrapper(dec_cell, 
                                             output_keep_prob=keep_prob)
    
    inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                        inference_helper,
                                                        encoder_state,
                                                        output_layer)
        
    # Perform dynamic decoding using the decoder
    inference_decoder_output = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                                                 impute_finished=True,
                                                                 maximum_iterations=max_target_sequence_length)[0]
    return inference_decoder_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [8]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
     # 1. Decoder Embedding
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)

    # 2. Construct the decoder cell(changed initializer)
    initializer = tf.random_uniform_initializer(-0.1, 0.1, seed=2)
    def make_cell(rnn_size, keep_prob):
        dec_cell = tf.contrib.rnn.LSTMCell(rnn_size,
                                           initializer=initializer)
        dec_cell = tf.contrib.rnn.DropoutWrapper(dec_cell, 
                                                 output_keep_prob=keep_prob)
        return dec_cell

    dec_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size, keep_prob) for _ in range(num_layers)])
     
    # 3. Dense layer to translate the decoder's output at each time 
    # step into a choice from the target vocabulary
    output_layer = Dense(target_vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    # 4. Training Decoder
    with tf.variable_scope("decode") as scope:
        training_decoder_output = decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                                                       target_sequence_length, max_target_sequence_length, 
                                                       output_layer, keep_prob)
    
    # 5. Inference Decoder(reuse variable)
        scope.reuse_variables()
        inference_decoder_output = decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, 
                                                        target_vocab_to_int['<GO>'],  target_vocab_to_int['<EOS>'], 
                                                        max_target_sequence_length, target_vocab_size, 
                                                        output_layer, batch_size, keep_prob)
    
    return training_decoder_output, inference_decoder_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [10]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    # get state from encoder
    _, enc_state = encoding_layer(input_data, rnn_size, num_layers, 
                                  keep_prob, source_sequence_length, 
                                  source_vocab_size, enc_embedding_size)
    
    # process decoder input data
    dec_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    
    # pass enc_state and dec_input to decoder
    training_decoder_output, inference_decoder_output = decoding_layer(dec_input, enc_state, 
                                                                       target_sequence_length, 
                                                                       max_target_sentence_length, 
                                                                       rnn_size, num_layers, 
                                                                       target_vocab_to_int, 
                                                                       target_vocab_size, 
                                                                       batch_size, keep_prob, 
                                                                       dec_embedding_size)
    return training_decoder_output, inference_decoder_output


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [9]:
# Number of Epochs
epochs = 50
# Batch Size
batch_size = 256
# RNN Size(change from 120)
rnn_size = 256
# Number of Layers
num_layers = 2
# Embedding Size(original 15)
encoding_embedding_size = 128
decoding_embedding_size = 128
# Learning Rate
learning_rate = 0.001
# Dropout Keep Probability
keep_probability = 0.75
display_step = 20

### Build the Graph
Build the graph using the neural network you implemented.

In [11]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [12]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [13]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch   20/538 - Train Accuracy: 0.3391, Validation Accuracy: 0.3844, Loss: 3.0046
Epoch   0 Batch   40/538 - Train Accuracy: 0.4783, Validation Accuracy: 0.4794, Loss: 2.2429
Epoch   0 Batch   60/538 - Train Accuracy: 0.4754, Validation Accuracy: 0.5192, Loss: 1.7886
Epoch   0 Batch   80/538 - Train Accuracy: 0.4506, Validation Accuracy: 0.5027, Loss: 1.3099
Epoch   0 Batch  100/538 - Train Accuracy: 0.5248, Validation Accuracy: 0.5526, Loss: 1.0050
Epoch   0 Batch  120/538 - Train Accuracy: 0.5289, Validation Accuracy: 0.5680, Loss: 0.8757
Epoch   0 Batch  140/538 - Train Accuracy: 0.5303, Validation Accuracy: 0.5865, Loss: 0.8637
Epoch   0 Batch  160/538 - Train Accuracy: 0.5952, Validation Accuracy: 0.5987, Loss: 0.7127
Epoch   0 Batch  180/538 - Train Accuracy: 0.6393, Validation Accuracy: 0.6309, Loss: 0.6760
Epoch   0 Batch  200/538 - Train Accuracy: 0.6516, Validation Accuracy: 0.6339, Loss: 0.6477
Epoch   0 Batch  220/538 - Train Accuracy: 0.6122, Validation Accuracy

Epoch   3 Batch  240/538 - Train Accuracy: 0.9533, Validation Accuracy: 0.9439, Loss: 0.0469
Epoch   3 Batch  260/538 - Train Accuracy: 0.9364, Validation Accuracy: 0.9508, Loss: 0.0570
Epoch   3 Batch  280/538 - Train Accuracy: 0.9552, Validation Accuracy: 0.9329, Loss: 0.0409
Epoch   3 Batch  300/538 - Train Accuracy: 0.9585, Validation Accuracy: 0.9490, Loss: 0.0509
Epoch   3 Batch  320/538 - Train Accuracy: 0.9474, Validation Accuracy: 0.9576, Loss: 0.0439
Epoch   3 Batch  340/538 - Train Accuracy: 0.9387, Validation Accuracy: 0.9464, Loss: 0.0461
Epoch   3 Batch  360/538 - Train Accuracy: 0.9463, Validation Accuracy: 0.9421, Loss: 0.0465
Epoch   3 Batch  380/538 - Train Accuracy: 0.9488, Validation Accuracy: 0.9371, Loss: 0.0376
Epoch   3 Batch  400/538 - Train Accuracy: 0.9444, Validation Accuracy: 0.9579, Loss: 0.0526
Epoch   3 Batch  420/538 - Train Accuracy: 0.9527, Validation Accuracy: 0.9432, Loss: 0.0413
Epoch   3 Batch  440/538 - Train Accuracy: 0.9527, Validation Accuracy

Epoch   6 Batch  460/538 - Train Accuracy: 0.9522, Validation Accuracy: 0.9595, Loss: 0.0432
Epoch   6 Batch  480/538 - Train Accuracy: 0.9676, Validation Accuracy: 0.9595, Loss: 0.0292
Epoch   6 Batch  500/538 - Train Accuracy: 0.9769, Validation Accuracy: 0.9471, Loss: 0.0269
Epoch   6 Batch  520/538 - Train Accuracy: 0.9605, Validation Accuracy: 0.9597, Loss: 0.0335
Epoch   7 Batch   20/538 - Train Accuracy: 0.9554, Validation Accuracy: 0.9558, Loss: 0.0342
Epoch   7 Batch   40/538 - Train Accuracy: 0.9682, Validation Accuracy: 0.9565, Loss: 0.0291
Epoch   7 Batch   60/538 - Train Accuracy: 0.9680, Validation Accuracy: 0.9611, Loss: 0.0365
Epoch   7 Batch   80/538 - Train Accuracy: 0.9656, Validation Accuracy: 0.9581, Loss: 0.0318
Epoch   7 Batch  100/538 - Train Accuracy: 0.9771, Validation Accuracy: 0.9558, Loss: 0.0256
Epoch   7 Batch  120/538 - Train Accuracy: 0.9730, Validation Accuracy: 0.9556, Loss: 0.0327
Epoch   7 Batch  140/538 - Train Accuracy: 0.9514, Validation Accuracy

Epoch  10 Batch  160/538 - Train Accuracy: 0.9635, Validation Accuracy: 0.9494, Loss: 0.0298
Epoch  10 Batch  180/538 - Train Accuracy: 0.9548, Validation Accuracy: 0.9544, Loss: 0.0408
Epoch  10 Batch  200/538 - Train Accuracy: 0.9719, Validation Accuracy: 0.9551, Loss: 0.0297
Epoch  10 Batch  220/538 - Train Accuracy: 0.9412, Validation Accuracy: 0.9411, Loss: 0.0423
Epoch  10 Batch  240/538 - Train Accuracy: 0.9625, Validation Accuracy: 0.9608, Loss: 0.0378
Epoch  10 Batch  260/538 - Train Accuracy: 0.9399, Validation Accuracy: 0.9624, Loss: 0.0433
Epoch  10 Batch  280/538 - Train Accuracy: 0.9658, Validation Accuracy: 0.9537, Loss: 0.0326
Epoch  10 Batch  300/538 - Train Accuracy: 0.9635, Validation Accuracy: 0.9627, Loss: 0.0359
Epoch  10 Batch  320/538 - Train Accuracy: 0.9673, Validation Accuracy: 0.9545, Loss: 0.0340
Epoch  10 Batch  340/538 - Train Accuracy: 0.9457, Validation Accuracy: 0.9480, Loss: 0.0404
Epoch  10 Batch  360/538 - Train Accuracy: 0.9654, Validation Accuracy

Epoch  13 Batch  380/538 - Train Accuracy: 0.9691, Validation Accuracy: 0.9537, Loss: 0.0284
Epoch  13 Batch  400/538 - Train Accuracy: 0.9468, Validation Accuracy: 0.9528, Loss: 0.0421
Epoch  13 Batch  420/538 - Train Accuracy: 0.9572, Validation Accuracy: 0.9576, Loss: 0.0335
Epoch  13 Batch  440/538 - Train Accuracy: 0.9637, Validation Accuracy: 0.9647, Loss: 0.0376
Epoch  13 Batch  460/538 - Train Accuracy: 0.9613, Validation Accuracy: 0.9695, Loss: 0.0390
Epoch  13 Batch  480/538 - Train Accuracy: 0.9719, Validation Accuracy: 0.9554, Loss: 0.0340
Epoch  13 Batch  500/538 - Train Accuracy: 0.9798, Validation Accuracy: 0.9586, Loss: 0.0247
Epoch  13 Batch  520/538 - Train Accuracy: 0.9617, Validation Accuracy: 0.9561, Loss: 0.0415
Epoch  14 Batch   20/538 - Train Accuracy: 0.9635, Validation Accuracy: 0.9583, Loss: 0.0417
Epoch  14 Batch   40/538 - Train Accuracy: 0.9608, Validation Accuracy: 0.9455, Loss: 0.0298
Epoch  14 Batch   60/538 - Train Accuracy: 0.9645, Validation Accuracy

...

Epoch  34 Batch  140/538 - Train Accuracy: 0.9375, Validation Accuracy: 0.9460, Loss: 0.0618
Epoch  34 Batch  160/538 - Train Accuracy: 0.9568, Validation Accuracy: 0.9567, Loss: 0.0443
Epoch  34 Batch  180/538 - Train Accuracy: 0.9470, Validation Accuracy: 0.9412, Loss: 0.0506
Epoch  34 Batch  200/538 - Train Accuracy: 0.9771, Validation Accuracy: 0.9387, Loss: 0.0389
Epoch  34 Batch  220/538 - Train Accuracy: 0.9433, Validation Accuracy: 0.9403, Loss: 0.0516
Epoch  34 Batch  240/538 - Train Accuracy: 0.9387, Validation Accuracy: 0.9419, Loss: 0.0512
Epoch  34 Batch  260/538 - Train Accuracy: 0.9087, Validation Accuracy: 0.9451, Loss: 0.0491
Epoch  34 Batch  280/538 - Train Accuracy: 0.9503, Validation Accuracy: 0.9545, Loss: 0.0407
Epoch  34 Batch  300/538 - Train Accuracy: 0.9461, Validation Accuracy: 0.9544, Loss: 0.0448
Epoch  34 Batch  320/538 - Train Accuracy: 0.9487, Validation Accuracy: 0.9462, Loss: 0.0438
Epoch  34 Batch  340/538 - Train Accuracy: 0.9459, Validation Accuracy

Epoch  37 Batch  360/538 - Train Accuracy: 0.9461, Validation Accuracy: 0.9538, Loss: 0.0409
Epoch  37 Batch  380/538 - Train Accuracy: 0.9637, Validation Accuracy: 0.9453, Loss: 0.0400
Epoch  37 Batch  400/538 - Train Accuracy: 0.9550, Validation Accuracy: 0.9489, Loss: 0.0461
Epoch  37 Batch  420/538 - Train Accuracy: 0.9588, Validation Accuracy: 0.9577, Loss: 0.0441
Epoch  37 Batch  440/538 - Train Accuracy: 0.9531, Validation Accuracy: 0.9556, Loss: 0.0437
Epoch  37 Batch  460/538 - Train Accuracy: 0.9485, Validation Accuracy: 0.9647, Loss: 0.0439
Epoch  37 Batch  480/538 - Train Accuracy: 0.9594, Validation Accuracy: 0.9618, Loss: 0.0371
Epoch  37 Batch  500/538 - Train Accuracy: 0.9608, Validation Accuracy: 0.9542, Loss: 0.0362
Epoch  37 Batch  520/538 - Train Accuracy: 0.9432, Validation Accuracy: 0.9540, Loss: 0.0432
Epoch  38 Batch   20/538 - Train Accuracy: 0.9500, Validation Accuracy: 0.9588, Loss: 0.0443
Epoch  38 Batch   40/538 - Train Accuracy: 0.9728, Validation Accuracy

Epoch  41 Batch   60/538 - Train Accuracy: 0.9547, Validation Accuracy: 0.9549, Loss: 0.0579
Epoch  41 Batch   80/538 - Train Accuracy: 0.9555, Validation Accuracy: 0.9387, Loss: 0.0570
Epoch  41 Batch  100/538 - Train Accuracy: 0.9402, Validation Accuracy: 0.9553, Loss: 0.0574
Epoch  41 Batch  120/538 - Train Accuracy: 0.9475, Validation Accuracy: 0.9407, Loss: 0.0473
Epoch  41 Batch  140/538 - Train Accuracy: 0.9137, Validation Accuracy: 0.9409, Loss: 0.0782
Epoch  41 Batch  160/538 - Train Accuracy: 0.9355, Validation Accuracy: 0.9302, Loss: 0.0489
Epoch  41 Batch  180/538 - Train Accuracy: 0.9412, Validation Accuracy: 0.9437, Loss: 0.0581
Epoch  41 Batch  200/538 - Train Accuracy: 0.9545, Validation Accuracy: 0.9503, Loss: 0.0437
Epoch  41 Batch  220/538 - Train Accuracy: 0.9304, Validation Accuracy: 0.9389, Loss: 0.0632
Epoch  41 Batch  240/538 - Train Accuracy: 0.9369, Validation Accuracy: 0.9414, Loss: 0.0518
Epoch  41 Batch  260/538 - Train Accuracy: 0.9317, Validation Accuracy

Epoch  44 Batch  280/538 - Train Accuracy: 0.9539, Validation Accuracy: 0.9501, Loss: 0.0523
Epoch  44 Batch  300/538 - Train Accuracy: 0.9541, Validation Accuracy: 0.9430, Loss: 0.0555
Epoch  44 Batch  320/538 - Train Accuracy: 0.9356, Validation Accuracy: 0.9265, Loss: 0.0583
Epoch  44 Batch  340/538 - Train Accuracy: 0.9350, Validation Accuracy: 0.9505, Loss: 0.0629
Epoch  44 Batch  360/538 - Train Accuracy: 0.9414, Validation Accuracy: 0.9428, Loss: 0.0620
Epoch  44 Batch  380/538 - Train Accuracy: 0.9324, Validation Accuracy: 0.9354, Loss: 0.0482
Epoch  44 Batch  400/538 - Train Accuracy: 0.9462, Validation Accuracy: 0.9446, Loss: 0.0581
Epoch  44 Batch  420/538 - Train Accuracy: 0.9461, Validation Accuracy: 0.9313, Loss: 0.0562
Epoch  44 Batch  440/538 - Train Accuracy: 0.9463, Validation Accuracy: 0.9377, Loss: 0.0643
Epoch  44 Batch  460/538 - Train Accuracy: 0.9235, Validation Accuracy: 0.9524, Loss: 0.0611
Epoch  44 Batch  480/538 - Train Accuracy: 0.9442, Validation Accuracy

Epoch  47 Batch  500/538 - Train Accuracy: 0.9627, Validation Accuracy: 0.9398, Loss: 0.0372
Epoch  47 Batch  520/538 - Train Accuracy: 0.9387, Validation Accuracy: 0.9347, Loss: 0.0556
Epoch  48 Batch   20/538 - Train Accuracy: 0.9422, Validation Accuracy: 0.9442, Loss: 0.0592
Epoch  48 Batch   40/538 - Train Accuracy: 0.9592, Validation Accuracy: 0.9528, Loss: 0.0428
Epoch  48 Batch   60/538 - Train Accuracy: 0.9461, Validation Accuracy: 0.9480, Loss: 0.0494
Epoch  48 Batch   80/538 - Train Accuracy: 0.9594, Validation Accuracy: 0.9412, Loss: 0.0551
Epoch  48 Batch  100/538 - Train Accuracy: 0.9574, Validation Accuracy: 0.9432, Loss: 0.0399
Epoch  48 Batch  120/538 - Train Accuracy: 0.9469, Validation Accuracy: 0.9419, Loss: 0.0409
Epoch  48 Batch  140/538 - Train Accuracy: 0.9189, Validation Accuracy: 0.9446, Loss: 0.0701
Epoch  48 Batch  160/538 - Train Accuracy: 0.9555, Validation Accuracy: 0.9444, Loss: 0.0452
Epoch  48 Batch  180/538 - Train Accuracy: 0.9488, Validation Accuracy

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [14]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [15]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [16]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    lc_sentence = sentence.lower()
    word_ids = [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in sentence.split()]
    
    return word_ids


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [17]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [113, 8, 110, 143, 177, 100, 52]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [117, 220, 253, 250, 199, 208, 45, 111, 1]
  French Words: il a vu un vieux camion bleu . <EOS>


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.